In [129]:
class RegresionLinealMultiple:  
    import statsmodels.formula.api as smf
    
    def __init__(self, url=None, y=None, **kwargs):
        self.url = url
        self.y = y
        self.kwargs = kwargs
        
    def variables_x(self):
        l = []
        for i in self.kwargs:
            l.append(i)
        return l
        
    def join_variables_x(self):
        var_x = self.variables_x()
        modelo_variables = '+'.join(var_x)
        return modelo_variables
    
    def modelo_multiple(self):
        
        import pandas as pd
        import statsmodels.formula.api as smf
        
        data = pd.read_csv(self.url)     
        variables_x = self.y + '~' + self.join_variables_x()
        lm = smf.ols(formula = variables_x, data = data).fit()
        return lm
    
    def view_model(self):
        #import statsmodels.formula.api as smf
        view = self.modelo_multiple()
        return view.summary()              
        
    def parameters_pvalues_rsquared(self):
        model = self.modelo_multiple()
        print('Parametros del modelo:\n{}\n-----------'.format(model.params))
        print('P_valores del modelo:\n{}\n-----------'.format(model.pvalues))
        print('R_cuadrada del modelo:\n{}\n-----------'.format(model.rsquared))
        print('R_cuadrada ajustada del modelo:\n{}\n-----------'.format(model.rsquared_adj))
        #print('Según el modelo: Sales = {} + {}*"TV" + {}*"Newspaper" '.format(model.params[0], model.params[1], model.params[2]))
       
    def prediccion_y(self):
        
        import pandas as pd
        
        """
         Llamamos a la función 'variables_x()' para obtener los nombre de
         las variables x y pasarlas por la función 'predict'.         
        """
         
        model = self.modelo_multiple()
        data = pd.read_csv(self.url)
        y_pred = model.predict(data[self.variables_x()])
        return y_pred
    
    def error(self):
        
        import pandas as pd
        import numpy as np
        
        count_var = len(self.variables_x())
        data = pd.read_csv(self.url) 
        SSD = sum((data[self.y]-self.prediccion_y())**2)
        RSE = np.sqrt(SSD/(len(data)-count_var-1))
        e = RSE / np.mean(data[self.y])
        # La salida de esta función, detalla el porcentaje de error que no puede explicar el modelo, entre menor sea es mejor para el modelo.
        return e
        

In [133]:
path = 'https://raw.githubusercontent.com/joanby/python-ml-course/master/datasets/ads/Advertising.csv'
regre_mult = RegresionLinealMultiple(url = path, y = 'Sales', TV = 'TV', Radio='Radio', Newspaper='Newspaper')

In [134]:
regre_mult.join_variables_x()

'TV+Radio+Newspaper'

In [136]:
regre_mult.view_model()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Sun, 09 Dec 2018   Prob (F-statistic):           1.58e-96
Time:                        16:53:55   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9389      0.312      9.422      0.000       2.324       3.554
TV             0.0458      0.001     32.809      0.000       0.043       0.049
Radio          0.1885      0.009     21.893      0.000       0.172       0.206
Newspaper     -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [138]:
regre_mult.error()

0.12020041885646236